In [1]:
import os
import json
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import pandas as pd

/home/bbc8731/miniconda3/envs/chatnetmedgpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open("../data/parameters.json", 'r') as file:
    param = json.load(file)
nodes = pd.read_csv(os.path.join(param['files']['data_dir'], "nodes_snake.csv"), sep= ',')
edges = pd.read_csv(os.path.join(param['files']['data_dir'], "edges.csv"), sep= ',')

In [9]:
edges.loc[(edges['node_types'] == "drug|disease") & (edges['relation'] == "contraindication"),:]

,relation,display_relation,x_index,y_index,node_types,xy,z_index
3676122,contraindication,contraindication,14268,38040,drug|disease,14268|38040,129388
3676124,contraindication,contraindication,16689,33605,drug|disease,16689|33605,129388
3676127,contraindication,contraindication,17730,38084,drug|disease,17730|38084,129388
3676128,contraindication,contraindication,14242,35958,drug|disease,14242|35958,129388
3676131,contraindication,contraindication,14560,36688,drug|disease,14560|36688,129388
...,...,...,...,...,...,...,...
3737460,contraindication,contraindication,14483,83819,drug|disease,14483|83819,129388
3737462,contraindication,contraindication,14235,30382,drug|disease,14235|30382,129388
3737463,contraindication,contraindication,14543,35641,drug|disease,14543|35641,129388
3737467,contraindication,contraindication,14791,36104,drug|disease,14791|36104,129388


In [ ]:
def meanpooling(outputs, attention_mask):
    token_embeddings = outputs.last_hidden_state          # [B, T, H]
    mask = attention_mask.unsqueeze(-1).float()           # [B, T, 1]
    summed = (token_embeddings * mask).sum(dim=1)         # [B, H]
    counts = mask.sum(dim=1).clamp(min=1e-9)              # [B, 1]
    return summed / counts


In [ ]:
# nodes

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("neuml/pubmedbert-base-embeddings")
model.eval()
model.to("cuda")  # or "cpu" if no GPU

def meanpooling(outputs, attention_mask):
    token_embeddings = outputs.last_hidden_state          # [B, T, H]
    mask = attention_mask.unsqueeze(-1).float()           # [B, T, 1]
    summed = (token_embeddings * mask).sum(dim=1)         # [B, H]
    counts = mask.sum(dim=1).clamp(min=1e-9)              # [B, 1]
    return summed / counts


def node_embedding(list_nodes_sentence, batch_size=32):
    all_embs = []

    for start in range(0, len(list_nodes_sentence), batch_size):
        batch = list_nodes_sentence[start : start + batch_size]

        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(model.device)

        with torch.no_grad():
            outputs = model(**inputs)

        emb = meanpooling(outputs, inputs["attention_mask"])
        all_embs.append(emb.cpu())

    return torch.cat(all_embs, dim=0)   # [N, 768]


# ------ your data ------
# list_nodes_sentence: list[str], one text per node
# all_node_names: list[str], same order as list_nodes_sentence

emb_all = node_embedding(list_nodes_sentence, batch_size=32)  # [N, 768]

# save embeddings + node names + index mapping
torch.save(emb_all, "node_emb_all.pt")
with open("all_node_names.json", "w") as f:
    json.dump(all_node_names, f)
name2id = {name: i for i, name in enumerate(all_node_names)}
with open("name2id.json", "w") as f:
    json.dump(name2id, f)
